In [1]:
import glob
import pandas as pd

In [2]:

files = glob.glob("../exp/simulation/remote_exp/remote_save_result/Rosenbrock_2d_total10:09:12:11:42/*")
for file in files:
    df = pd.read_csv(file)

In [3]:
df

,1,2,4,5,10,20
0,-3.93475,-3.974885,-0.715053,-4.32753,-3.465912,-1.758212


In [4]:
import os, sys

import numpy as np

from ML2_lib import loss
from ML2_lib import RV_SGDAve

import pandas as pd
import datetime
from tqdm import tqdm
args = sys.argv
d = 2
trial_num = 2
lr = 0.001

c = 1

n = 100
k_list = [0, 1, 3, 4, 9, 19]
# ノイズの種類（勾配）
noise_type_list = ["lognormal", "normal", "student_t"]

E_var = 1.75
# ノイズの分散（勾配）
def noise_var(noise_type):
    if noise_type == "lognormal":
        return [1.2,1.75,1.9]
    elif noise_type == "normal":
        return [1.5,1.9,2.2]
    elif noise_type == "student_t":
        return [1.5,3,5]
    else:
        raise ValueError("lognormalかnormalかsutdent_tで")


# 検証時に加える関数値にノイズの種類
noise_type_f = "lognormal"
f_E_var = 1.75

# 初期値
w_init_list_0 = [-10, 0, 10]
w_init_list_1 = [-10, 0, 10]

k_string = [f"{i + 1}" for i in k_list]

now = datetime.datetime.now()


for noise_type in tqdm(noise_type_list):
    noise_var_list = noise_var(noise_type)
    for E_var in noise_var_list:
        for w_init_0 in w_init_list_0:
            for w_init_1 in w_init_list_1:
                son = loss.RosenBrock(d=d, noise_type=noise_type, E_var=E_var, f_E_var=f_E_var, noise_type_f=noise_type_f)
                RV = RV_SGDAve.RVSGDByW(model_opt=son, c=c, n=n, lr=lr)
                w_init = np.array([w_init_0, w_init_1])
                _, result = RV.many_trails(trial_num=trial_num, max_k=k_list[-1] + 1, w_init=w_init)
                title = f"RVSGD Rosenbrock trial = {trial_num} noise = {noise_type} D = {d}_sample_num{n} noise var = {E_var}　w_init = {w_init} f_noise{noise_type_f} f_noise_E{f_E_var}"
                df = pd.DataFrame(result[:, k_list], columns=k_string)


100%|██████████| 3/3 [01:10<00:00, 23.34s/it]


In [5]:

df

,1,2,4,5,10,20
0,106450.007616,225323.185815,396943.287135,441920.269514,600091.403483,746977.353001
1,106447.913778,225326.017718,396944.036690,441921.153495,600091.261678,746976.248069
